In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
import os

# --- CONFIGURAÇÃO ---
BASE_PATH = "../data/raw/CNPJ"
ARQUIVOS_ESTAB = [os.path.join(BASE_PATH, f"Estabelecimentos{i}/K3241.K03200Y{i}.D50913.ESTABELE") for i in range(10)]
CAMINHO_SIMPLES = os.path.join(BASE_PATH, "Simples/F.K03200$W.SIMPLES.CSV.D50913")
SAIDA_FINAL = "../data/processed/painel_mei_rf_anual.parquet"

def main():
    print("--- INICIANDO ETL MEI OTIMIZADO ---")
    
    # 1. Carregar Simples com tipos otimizados
    # O CNPJ Básico deve ser string ou int para o merge bater
    df_simples = pd.read_csv(CAMINHO_SIMPLES, sep=";", encoding="latin-1", header=None, 
                             usecols=[0, 4, 5, 6], names=["cnpj_basico", "mei", "ini", "fim"],
                             dtype={"cnpj_basico": "int32", "mei": "category"})
    
    df_simples = df_simples[df_simples["mei"] == "S"].copy()
    
    # Datas para Ano (Vetorizado) - convertemos para int16 para economizar RAM
    df_simples['ano_ini'] = (df_simples['ini'] // 10000).fillna(0).astype("int16")
    df_simples['ano_fim'] = (df_simples['fim'] // 10000).fillna(0).astype("int16")
    df_simples.drop(columns=["mei", "ini", "fim"], inplace=True)

    painel_final_lista = []

    # 2. Loop Estabelecimentos
    for path in ARQUIVOS_ESTAB:
        if not os.path.exists(path): continue
        
        # Lista temporária por ARQUIVO para evitar fragmentação global
        painel_arquivo = []
        
        reader = pd.read_csv(path, sep=";", encoding="latin-1", header=None,
                             usecols=[0, 11, 20], names=["cnpj_basico", "cnae", "mun"],
                             dtype={"cnpj_basico": "int32", "cnae": "float64", "mun": "float64"},
                             chunksize=2_000_000)
        
        for chunk in tqdm(reader, desc=f"📂 {os.path.basename(path)}"):
            # Merge Inner (remove quem não é MEI logo de cara)
            chunk = chunk.merge(df_simples, on="cnpj_basico", how="inner")
            
            # Setor Vetorizado (Muito mais rápido que .apply)
            # CNAE na RF costuma ter 7 dígitos. Divisão são os 2 primeiros.
            divisao = (chunk['cnae'] // 100000).fillna(0).astype(int)
            chunk['setor'] = 'Servicos'
            chunk.loc[divisao.between(1, 3), 'setor'] = 'Agro'
            chunk.loc[divisao.between(5, 33), 'setor'] = 'Industria'
            chunk.loc[divisao == 84, 'setor'] = 'Setor Publico'
            
            # Agregação rápida
            ent = chunk.groupby(['mun', 'setor', 'ano_ini']).size().reset_index(name='entradas')
            sai = chunk.dropna(subset=['ano_fim']).groupby(['mun', 'setor', 'ano_fim']).size().reset_index(name='saidas')
            
            painel_arquivo.append(pd.merge(ent.rename(columns={'ano_ini': 'ano'}), 
                                           sai.rename(columns={'ano_fim': 'ano'}), 
                                           on=['mun', 'setor', 'ano'], how='outer').fillna(0))
            
        # Consolida o arquivo atual e limpa a RAM antes do próximo
        if painel_arquivo:
            df_arq = pd.concat(painel_arquivo).groupby(['mun', 'setor', 'ano']).sum().reset_index()
            painel_final_lista.append(df_arq)
            del painel_arquivo; gc.collect()

    # 3. Consolidação e Cálculo de Estoque
    df_final = pd.concat(painel_final_lista).groupby(['mun', 'setor', 'ano']).sum().reset_index()
    
    # Ordenar para o cumsum fazer sentido (Econometria de Painel)
    df_final = df_final.sort_values(['mun', 'setor', 'ano'])
    df_final['estoque_mei'] = (
        df_final.groupby(['mun', 'setor'])['entradas'].cumsum() - 
        df_final.groupby(['mun', 'setor'])['saidas'].cumsum()
    )
    df_final.to_parquet(SAIDA_FINAL, index=False)
    print(f"✅ Painel concluído: {SAIDA_FINAL}")

if __name__ == "__main__":
    main()

--- INICIANDO ETL MEI OTIMIZADO ---


📂 K3241.K03200Y0.D50913.ESTABELE: 13it [19:51, 91.67s/it] 
📂 K3241.K03200Y1.D50913.ESTABELE: 3it [15:58, 319.50s/it]
📂 K3241.K03200Y2.D50913.ESTABELE: 3it [17:05, 342.00s/it]
📂 K3241.K03200Y3.D50913.ESTABELE: 3it [17:37, 352.41s/it]
📂 K3241.K03200Y4.D50913.ESTABELE: 3it [00:29,  9.75s/it]
📂 K3241.K03200Y5.D50913.ESTABELE: 3it [00:25,  8.51s/it]
📂 K3241.K03200Y6.D50913.ESTABELE: 3it [17:24, 348.11s/it]
📂 K3241.K03200Y7.D50913.ESTABELE: 3it [16:02, 320.68s/it]
📂 K3241.K03200Y8.D50913.ESTABELE: 3it [15:02, 300.99s/it]
📂 K3241.K03200Y9.D50913.ESTABELE: 3it [00:55, 18.38s/it]


✅ Painel concluído: ../data/processed/painel_mei_rf_anual.parquet


In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
import os
from pathlib import Path

# --- CONFIGURAÇÃO DE CAMINHOS ---
BASE_PATH = "../data/raw/CNPJ"
# Lista os 10 arquivos de estabelecimentos conforme o layout da RFB [cite: 3, 16]
ARQUIVOS_ESTAB = [os.path.join(BASE_PATH, f"Estabelecimentos{i}/K3241.K03200Y{i}.D50913.ESTABELE") for i in range(10)]
CAMINHO_SIMPLES = os.path.join(BASE_PATH, "Simples/F.K03200$W.SIMPLES.CSV.D50913")
OUTPUT_DIR = "../data/processed"
SAIDA_FINAL = os.path.join(OUTPUT_DIR, 'painel_mei_rf_anual.parquet')

def main():
    print("--- INICIANDO ETL MEI OTIMIZADO E CORRIGIDO ---")
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    # 1. Carregar Simples: Contém os indicadores de opção pelo MEI e as datas [cite: 20]
    print("🚀 Carregando base do Simples...")
    df_simples = pd.read_csv(
        CAMINHO_SIMPLES, 
        sep=";", 
        encoding="latin-1", 
        header=None, 
        usecols=[0, 4, 5, 6], 
        names=["cnpj_basico", "opcao_mei", "data_ini", "data_fim"],
        dtype={"cnpj_basico": "int32", "opcao_mei": "str", "data_ini": "str", "data_fim": "str"}
    )
    
    # Filtra apenas quem é ou já foi optante pelo MEI [cite: 20]
    df_simples = df_simples[df_simples["opcao_mei"] == "S"].copy()
    
    # Converte datas para Ano via vetorização (YYYYMMDD -> YYYY) 
    df_simples['ano_ini'] = pd.to_numeric(df_simples['data_ini'], errors='coerce').fillna(0) // 10000
    df_simples['ano_fim'] = pd.to_numeric(df_simples['data_fim'], errors='coerce').fillna(0) // 10000
    
    # Limpeza: Mantemos apenas anos razoáveis para evitar o "ano zero"
    df_simples = df_simples.drop(columns=["opcao_mei", "data_ini", "data_fim"])
    gc.collect()

    painel_final_lista = []

    # 2. Loop Estabelecimentos: Contém CNAE e Código de Município [cite: 3, 17]
    for path in ARQUIVOS_ESTAB:
        if not os.path.exists(path):
            continue
        
        painel_arquivo = []
        # Chunksize grande para velocidade, mas monitorando a RAM [cite: 38, 39]
        reader = pd.read_csv(
            path, 
            sep=";", 
            encoding="latin-1", 
            header=None,
            usecols=[0, 11, 20], 
            names=["cnpj_basico", "cnae", "mun"],
            dtype={"cnpj_basico": "int32", "cnae": "str", "mun": "float64"},
            chunksize=2_000_000
        )
        
        for chunk in tqdm(reader, desc=f"📂 {os.path.basename(path)}"):
            # Cruzamento interno: só processamos quem está na lista de MEIs [cite: 5, 11]
            chunk = chunk.merge(df_simples, on="cnpj_basico", how="inner")
            
            # Mapeamento Setorial (Divisão CNAE - 2 primeiros dígitos) [cite: 17, 36]
            div = chunk['cnae'].str[:2].fillna('00')
            chunk['setor'] = 'Servicos'
            chunk.loc[div.isin(['01', '02', '03']), 'setor'] = 'Agro'
            chunk.loc[div.astype(int).between(5, 33), 'setor'] = 'Industria'
            chunk.loc[div == '84', 'setor'] = 'Setor Publico'
            
            # Agregação de Entradas: Apenas anos válidos (> 1900)
            ent = chunk[chunk['ano_ini'] > 1900].groupby(['mun', 'setor', 'ano_ini']).size().reset_index(name='entradas')
            ent.columns = ['mun', 'setor', 'ano', 'entradas']
            
            # Agregação de Saídas: CRUCIAL - Ignora NaTs/Zeros para evitar estoque negativo
            sai = chunk[chunk['ano_fim'] > 1900].groupby(['mun', 'setor', 'ano_fim']).size().reset_index(name='saidas')
            sai.columns = ['mun', 'setor', 'ano', 'saidas']
            
            # Merge de Fluxo
            fluxo = pd.merge(ent, sai, on=['mun', 'setor', 'ano'], how='outer').fillna(0)
            painel_arquivo.append(fluxo)
            
            del chunk, ent, sai, fluxo
            gc.collect()
            
        if painel_arquivo:
            # Consolida o arquivo atual antes de ir para o próximo (previne fragmentação)
            df_arq = pd.concat(painel_arquivo).groupby(['mun', 'setor', 'ano']).sum().reset_index()
            painel_final_lista.append(df_arq)
            del painel_arquivo
            gc.collect()

    # 3. Consolidação e Cálculo de Estoque
    print("\n📊 Gerando painel final e calculando estoques...")
    df_final = pd.concat(painel_final_lista).groupby(['mun', 'setor', 'ano']).sum().reset_index()
    
    # Ordenação necessária para o cálculo acumulado (Time-series)
    df_final = df_final.sort_values(['mun', 'setor', 'ano'])
    
    # Estoque = Soma acumulada de quem entrou (-) soma acumulada de quem saiu
    df_final['estoque_mei'] = (
        df_final.groupby(['mun', 'setor'])['entradas'].cumsum() - 
        df_final.groupby(['mun', 'setor'])['saidas'].cumsum()
    )

    # Exportação em Parquet para performance no modelo econométrico
    df_final.to_parquet(SAIDA_FINAL, index=False)
    print(f"✅ Painel concluído com sucesso: {SAIDA_FINAL}")

if __name__ == "__main__":
    main()

--- INICIANDO ETL MEI OTIMIZADO E CORRIGIDO ---
🚀 Carregando base do Simples...


📂 K3241.K03200Y0.D50913.ESTABELE: 13it [02:33, 11.80s/it]
📂 K3241.K03200Y1.D50913.ESTABELE: 3it [01:05, 21.98s/it]
📂 K3241.K03200Y2.D50913.ESTABELE: 3it [01:04, 21.38s/it]
📂 K3241.K03200Y3.D50913.ESTABELE: 3it [01:08, 22.73s/it]
📂 K3241.K03200Y4.D50913.ESTABELE: 3it [00:58, 19.61s/it]
📂 K3241.K03200Y5.D50913.ESTABELE: 3it [00:46, 15.44s/it]
📂 K3241.K03200Y6.D50913.ESTABELE: 3it [00:48, 16.11s/it]
📂 K3241.K03200Y7.D50913.ESTABELE: 3it [00:49, 16.62s/it]
📂 K3241.K03200Y8.D50913.ESTABELE: 3it [00:47, 15.92s/it]
📂 K3241.K03200Y9.D50913.ESTABELE: 3it [00:48, 16.29s/it]



📊 Gerando painel final e calculando estoques...
✅ Painel concluído com sucesso: ../data/processed\painel_mei_rf_anual.parquet


In [ ]:
import pandas as pd
import numpy as np
import os
import gc

# --- CONFIGURAÇÃO DE CAMINHOS ---
INPUT_FILE = '../data/raw/test.parquet'
OUTPUT_DIR = '../data/processed'
OUTPUT_FILE = os.path.join(OUTPUT_DIR, 'rais_painel_balanceado.parquet')

def processar_rais_otimizado(path):
    """
    Lê apenas colunas necessárias e usa vetorização para economizar RAM.
    """
    print("   [1/3] Lendo colunas selecionadas e mapeando setores...")
    
    # Lê apenas o necessário
    cols = ['id_municipio', 'cnae_2', 'ano', 'quantidade_vinculos_ativos']
    df = pd.read_parquet(path, columns=cols)

    # Vetorização em vez de apply: muito mais rápido e leve
    # CNAE Divisão (2 dígitos)
    div = (df['cnae_2'].fillna(0).astype(np.int32) // 1000)
    
    df['setor'] = 'Outros'
    df.loc[div.between(1, 3), 'setor'] = 'Agro'
    df.loc[div.between(5, 33), 'setor'] = 'Industria'
    df.loc[div == 84, 'setor'] = 'Setor Publico'
    df.loc[div.between(35, 99), 'setor'] = 'Servicos'
    
    # Remove coluna temporária e libera memória
    df.drop(columns=['cnae_2'], inplace=True)

    print("   [2/3] Agregando estoque...")
    df_agg = (df.groupby(['id_municipio', 'setor', 'ano'], as_index=False)
                ['quantidade_vinculos_ativos'].sum())
    
    del df
    gc.collect()
    return df_agg

def rebalancear_painel_otimizado(df_agg):
    """
    Usa tipos de dados categóricos para reduzir o tamanho do produto cartesiano.
    """
    print("   [3/3] Rebalanceando o painel...")

    # Remover 'Outros' antes de balancear economiza memória
    df_agg = df_agg[df_agg['setor'] != 'Outros'].copy()

    todos_municipios = df_agg['id_municipio'].unique()
    todos_setores = ['Agro', 'Industria', 'Servicos', 'Setor Publico']
    todos_anos = df_agg['ano'].unique()

    # Criar index e esqueleto
    index = pd.MultiIndex.from_product(
        [todos_municipios, todos_setores, todos_anos], 
        names=['id_municipio', 'setor', 'ano']
    )
    
    # Reindex é mais eficiente que Merge para balanceamento
    df_final = (df_agg.set_index(['id_municipio', 'setor', 'ano'])
                      .reindex(index, fill_value=0)
                      .reset_index())

    # Variável para DID
    df_final['log_estoque'] = np.log1p(df_final['quantidade_vinculos_ativos'].astype(np.float32))
    
    return df_final

def main():
    print("--- INICIANDO ETL RAIS OTIMIZADO ---")
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    if not os.path.exists(INPUT_FILE):
        print(f"ERRO: Arquivo de entrada não encontrado: {INPUT_FILE}")
        return

    # Processamento em etapas com limpeza de cache
    df_agg = processar_rais_otimizado(INPUT_FILE)
    df_final = rebalancear_painel_otimizado(df_agg)
    
    print(f"Salvando em: {OUTPUT_FILE}")
    # compressão snappy é padrão e rápida
    df_final.to_parquet(OUTPUT_FILE, index=False, compression='snappy')
    print("--- CONCLUÍDO ---")

if __name__ == "__main__":
    main()

In [1]:
import pandas as pd

def get_simples_base(path, dtypes):
    df = pd.read_csv(path, sep=";", encoding="latin-1", header=None, 
                     usecols=[0, 4, 5, 6], names=["cnpj_basico", "mei", "ini", "fim"],
                     dtype=dtypes)
    df = df[df["mei"] == "S"].copy()
    df['ano_ini'] = (df['ini'] // 10000).fillna(0).astype("int16")
    df['ano_fim'] = (df['fim'] // 10000).fillna(0).astype("int16")
    return df.drop(columns=["mei", "ini", "fim"])

def classify_sector(df):
    divisao = (df['cnae'] // 100000).fillna(0).astype(int)
    df['setor'] = 'Servicos'
    df.loc[divisao.between(1, 3), 'setor'] = 'Agro'
    df.loc[divisao.between(5, 33), 'setor'] = 'Industria'
    df.loc[divisao == 84, 'setor'] = 'Setor Publico'
    return df

In [9]:
import basedosdados as bd

billing_id = "dissertacao-pnate"

query = """
SELECT
    dados.ano,
    dados.sigla_uf,
    dados.id_municipio,
    diretorio_id_municipio.nome AS municipio_nome,
    dic1.valor AS grau_instrucao_1985_2005,
    dic2.valor AS grau_instrucao_apos_2005,
    COUNT(*) AS total_vinculos
FROM `basedosdados.br_me_rais.microdados_vinculos` AS dados TABLESAMPLE SYSTEM (5 PERCENT)
LEFT JOIN `basedosdados.br_bd_diretorios_brasil.municipio` AS diretorio_id_municipio
    ON dados.id_municipio = diretorio_id_municipio.id_municipio
LEFT JOIN `basedosdados.br_me_rais.dicionario` AS dic1
    ON dados.grau_instrucao_1985_2005 = dic1.chave 
    AND dic1.nome_coluna = 'grau_instrucao_1985_2005' 
    AND dic1.id_tabela = 'microdados_vinculos'
LEFT JOIN `basedosdados.br_me_rais.dicionario` AS dic2
    ON dados.grau_instrucao_apos_2005 = dic2.chave 
    AND dic2.nome_coluna = 'grau_instrucao_apos_2005' 
    AND dic2.id_tabela = 'microdados_vinculos'
WHERE dados.ano >= 1990
  AND dados.sigla_uf IN ('AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE')
GROUP BY 1, 2, 3, 4, 5, 6
ORDER BY ano, id_municipio
"""

df = bd.read_sql(query=query, billing_project_id=billing_id)

Downloading: 100%|██████████|


In [11]:
# 1. Cria a coluna unificada pegando o valor que existir
df['escolaridade'] = df['grau_instrucao_1985_2005'].fillna(df['grau_instrucao_apos_2005'])

# 2. Remove as colunas antigas para limpar o df
df.drop(columns=['grau_instrucao_1985_2005', 'grau_instrucao_apos_2005'], inplace=True)

# 3. Agrupa novamente para consolidar as linhas (necessário se os IDs eram diferentes)
df = df.groupby(['ano', 'sigla_uf', 'id_municipio', 'municipio_nome', 'escolaridade'], as_index=False)['total_vinculos'].sum()